In [1]:
safety <- read.csv("train.csv")
safety$Choice[safety$Ch1=="1"] <- 0
safety$Choice[safety$Ch2=="1"] <- 1
safety$Choice[safety$Ch3=="1"] <- 2
safety$Choice[safety$Ch4=="1"] <- 3

In [2]:
# install.packages("XML")
library("xgboost")  # the main algorithm
library("archdata") # for the sample dataset
library("caret")    # for the confusionmatrix() function (also needs e1071 package)
library("dplyr")    # for some data preperation
library("Ckmeans.1d.dp") # for xgb.ggplot.importance

Loading required package: lattice
Loading required package: ggplot2
Warning message:
“package ‘dplyr’ was built under R version 3.5.1”
Attaching package: ‘dplyr’

The following object is masked from ‘package:xgboost’:

    slice

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [3]:
# function to calculate multinomial logloss
logLoss <- function(pred, actual){
  -1*mean(log(pred[model.matrix(~ actual + 0) - pred > 0]))
}

# original_vector<-c('Case','No','Task','CC1','GN1','NS1','BU1', 'FA1' ,'LD1', 'BZ1', 'FC1', 'FP1', 'RP1' ,'PP1' ,
#                 'KA1', 'SC1', 'TS1', 'NV1', 'MA1', 'LB1' ,'AF1' ,'HU1' ,'Price1' ,'CC2' ,'GN2', 'NS2', 'BU2' ,
#                 'FA2', 'LD2', 'BZ2', 'FC2', 'FP2', 'RP2' ,'PP2', 'KA2' ,'SC2' ,'TS2' ,'NV2', 'MA2' ,'LB2', 'AF2' ,
#                 'HU2', 'Price2' ,'CC3' ,'GN3', 'NS3', 'BU3' ,'FA3', 'LD3' ,'BZ3' ,'FC3', 'FP3', 'RP3', 'PP3' ,'KA3',
#                 'SC3' ,'TS3' ,'NV3' ,'MA3' ,'LB3', 'AF3', 'HU3' ,'Price3', 'CC4', 'GN4', 'NS4', 'BU4' ,'FA4', 'LD4',
#                 'BZ4' ,'FC4' ,'FP4' ,'RP4', 'PP4' ,'KA4', 'SC4' ,'TS4' ,'NV4', 'MA4', 'LB4', 'AF4' ,'HU4', 'Price4',
#                 'segmentind' , 'yearind','milesind','milesa','nightind','nighta' , 'pparkind','genderind', 'ageind',
#                 'agea','educind', 'regionind', 'Urbind', 'incomeind', 'incomea' , 'Choice')
# dat <- safety[c('Case','No','Task','CC1','GN1'  ,'BU1', 'FA1'     , 'BZ1', 'FP1', 'RP1' ,'PP1' ,
#                 'KA1', 'SC1', 'TS1', 'NV1', 'LB1' ,'AF1' ,'HU1' ,'Price1' ,'CC2' , 'NS2', 'BU2' ,
#                 'FA2', 'LD2', 'BZ2', 'FP2', 'RP2' ,'PP2' ,'SC2' ,'TS2' ,'NV2', 'MA2'     , 'AF2' ,
#                 'HU2', 'Price2' ,'CC3' ,'GN3'   , 'BU3' ,'FA3', 'LD3' ,'BZ3' ,'FC3', 'FP3', 'RP3', 'PP3' ,'KA3',
#                 'SC3' ,'TS3' ,'NV3' ,'MA3' ,'LB3', 'AF3', 'HU3' ,'Price3', 'CC4', 'GN4', 'NS4', 'BU4' ,'FA4', 'LD4',
#                 'BZ4' ,'FC4' ,'FP4' ,'RP4', 'PP4' ,'KA4', 'SC4' ,'TS4' ,'NV4', 'MA4', 'LB4', 'AF4' ,'HU4', 'Price4',
#                 'segmentind' ,'year', 'yearind','milesa','nighta' , 'pparkind','genderind', 
#                 'agea','educind', 'regionind', 'Urbind', 'incomea' , 'Choice')]

In [4]:
# dat <- safety[c(1:83,85:87,89,90,92,93,95,97,99,100,102,104,106,108,109,114)]
dat <-safety[c('Case','No','Task','CC1','GN1','NS1','BU1', 'FA1' ,'LD1', 'BZ1', 'FC1', 'FP1', 'RP1' ,'PP1' ,
                'KA1', 'SC1', 'TS1', 'NV1', 'MA1', 'LB1' ,'AF1' ,'HU1' ,'Price1' ,'CC2' ,'GN2', 'NS2', 'BU2' ,
                'FA2', 'LD2', 'BZ2', 'FC2', 'FP2', 'RP2' ,'PP2', 'KA2' ,'SC2' ,'TS2' ,'NV2', 'MA2' ,'LB2', 'AF2' ,
                'HU2', 'Price2' ,'CC3' ,'GN3', 'NS3', 'BU3' ,'FA3', 'LD3' ,'BZ3' ,'FC3', 'FP3', 'RP3', 'PP3' ,'KA3',
                'SC3' ,'TS3' ,'NV3' ,'MA3' ,'LB3', 'AF3', 'HU3' ,'Price3', 'CC4', 'GN4', 'NS4', 'BU4' ,'FA4', 'LD4',
                'BZ4' ,'FC4' ,'FP4' ,'RP4', 'PP4' ,'KA4', 'SC4' ,'TS4' ,'NV4', 'MA4', 'LB4', 'AF4' ,'HU4', 'Price4',
                'segmentind' , 'yearind','milesind','milesa','nightind','nighta' , 'pparkind','genderind', 'ageind',
                'agea','educind', 'regionind', 'Urbind', 'incomeind', 'incomea' , 'Choice')]
# data.train.small <- subset(safety, Task<=12)
data.train.small <- subset(dat, Case<=600)
# import validation set
# data.valid <- subset(safety, Task>=13) #nrow=5250
data.valid <- subset(safety, Case>600) #nrow=2850
# import test set
data.test <- read.csv("test.csv")
# NROW(data.valid)
# NCOL(dat)

In [5]:
head(dat)

Case,No,Task,CC1,GN1,NS1,BU1,FA1,LD1,BZ1,⋯,pparkind,genderind,ageind,agea,educind,regionind,Urbind,incomeind,incomea,Choice
1,1,1,0,1,4,6,0,0,0,⋯,1,1,4,53,5,2,2,3,44500,1
1,2,2,3,0,0,0,2,2,2,⋯,1,1,4,53,5,2,2,3,44500,0
1,3,3,1,1,5,0,2,0,3,⋯,1,1,4,53,5,2,2,3,44500,1
1,4,4,0,0,0,2,0,3,0,⋯,1,1,4,53,5,2,2,3,44500,1
1,5,5,0,2,2,0,1,2,0,⋯,1,1,4,53,5,2,2,3,44500,1
1,6,6,3,0,0,5,0,0,1,⋯,1,1,4,53,5,2,2,3,44500,1


In [6]:
# Make split index
# train_index <- sample(1:nrow(dat), nrow(dat)*0.75)
train_index <- (1:11400)
data_variables <- as.matrix(subset(dat, select= -c(Case,No,Task,Choice))) 
data_label <- dat[,"Choice"]

In [7]:
data_matrix <- xgb.DMatrix(data = as.matrix(dat), label = data_label)
# split train data and make xgb.DMatrix
train_data   <- data_variables[train_index,]
train_label  <- data_label[train_index]
train_matrix <- xgb.DMatrix(data = train_data, label = train_label)
# split test data and make xgb.DMatrix
test_data  <- data_variables[-train_index,]
test_label <- data_label[-train_index]
test_matrix <- xgb.DMatrix(data = test_data, label = test_label)

In [8]:
xgb_params <- list("objective" = "multi:softprob",
                   "eval_metric" = "mlogloss",
                   "num_class" = 4,'eta'=0.001,alpha=5,max_depth = 20,nthread=3) #,'eta'=0.2
nround    <- 4000 # number of XGBoost rounds
cv.nfold  <- 10

# Fit cv.nfold * cv.nround XGB models and save OOF predictions
# cv_model <- xgb.cv(params = xgb_params,
#                    data = train_matrix, 
#                    nrounds = nround,
#                    nfold = cv.nfold,
#                    verbose = FALSE,
#                    prediction = TRUE,print.every.n = 100)

In [9]:
# OOF_prediction <- data.frame(cv_model$pred) %>%
#   mutate(max_prob = max.col(., ties.method = "last"),
#          label = train_label + 1)
# head(OOF_prediction)

ERROR: Error in data.frame(cv_model$pred): object 'cv_model' not found


In [10]:
logLoss(OOF_prediction[,c(1:4)],OOF_prediction$label)
#boost=50, fold=5 1.84116935344035
#boost=100, fold=10 2.05634656583104 
#boost=100, fold=5 2.08338357311609
#boost=80, fold=5 1.98991888707679
#boost=50, fold=10 1.83534185255931
#boost=20, fold=10 1.66667567864521
#boost=5, fold=10 1.4759499524694 
#boost=1, fold=10 1.39543453666543 0.4635
#boost=5, fold=10 1.38629457338748 0.4683 'eta'=0.003,alpha=1 too conservative
#boost=5, fold=10 1.43521209844477 0.4932 'eta'=0.2,alpha=1
#boost=1000, fold=10 1.55922221482964 0.5553 'eta'=0.003,alpha=5,maxdepth=10 about 1 hour

ERROR: Error in mean(log(pred[model.matrix(~actual + 0) - pred > 0])): object 'OOF_prediction' not found


In [ ]:
confusionMatrix(factor(OOF_prediction$max_prob),
                factor(OOF_prediction$label),
                mode = "everything")

### Train Full Model and Assess Test Set Error

In [11]:
bst_model <- xgb.train(params = xgb_params,
                       data = train_matrix,
                       nrounds = nround,verbose = FALSE,print_every_n = 100)

#started at 2330, end at 0020
#started at 0025, end at 

ERROR: Error in matrix(test_pred, nrow = numberOfClasses, ncol = length(test_pred)/numberOfClasses): object 'numberOfClasses' not found


In [12]:
# Predict hold-out test set
test_pred <- predict(bst_model, newdata = test_matrix)
test_prediction <- matrix(test_pred, nrow = 4,
                          ncol=length(test_pred)/4) %>%
  t() %>%
  data.frame() %>%
  mutate(label = test_label + 1,
         max_prob = max.col(., "last"))
# confusion matrix of test set
confusionMatrix(factor(test_prediction$max_prob),
                factor(test_prediction$label),
                mode = "everything")

Confusion Matrix and Statistics

          Reference
Prediction   1   2   3   4
         1 296  77  84 138
         2 112 392 116 198
         3  79  87 304 152
         4 160 143 130 382

Overall Statistics
                                          
               Accuracy : 0.4821          
                 95% CI : (0.4636, 0.5006)
    No Information Rate : 0.3053          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.3061          
 Mcnemar's Test P-Value : 0.0008001       

Statistics by Class:

                     Class: 1 Class: 2 Class: 3 Class: 4
Sensitivity            0.4575   0.5608   0.4795   0.4391
Specificity            0.8643   0.8020   0.8565   0.7813
Pos Pred Value         0.4975   0.4792   0.4887   0.4687
Neg Pred Value         0.8443   0.8489   0.8519   0.7602
Precision              0.4975   0.4792   0.4887   0.4687
Recall                 0.4575   0.5608   0.4795   0.4391
F1                     0.4

In [13]:
head(test_prediction)
logLoss(test_prediction[,c(1:4)],test_prediction$label)
# 1.38629458008731
#nrounds=1000 1.54511870127743 
#nrounds=4000 1.5848540447609

X1,X2,X3,X4,label,max_prob
0.14459905,0.14532855,0.61468548,0.09538695,3,3
0.09164417,0.69487989,0.07945108,0.13402487,2,2
0.13957034,0.09824861,0.12409408,0.63808692,4,4
0.09830456,0.33668357,0.18008678,0.38492507,2,4
0.09456966,0.38244921,0.07798176,0.44499940,4,4
0.10063786,0.06057533,0.65492260,0.18386424,3,3


[1] 1.587383

In [ ]:
# get the feature real names
names <-  colnames(dat[,-c(1,2,3)])
# compute feature importance matrix
importance_matrix = xgb.importance(model = bst_model)
NROW(importance_matrix)
tail(importance_matrix)

In [ ]:
# plot
gp = xgb.ggplot.importance(importance_matrix)
print(gp) 

In [ ]:
# removed_vector<-c('NS1','LB2','LD1','FC1','MA1','KA2','GN2','FC2','NS3','incomeind','milesind','nightind','ageind')

In [ ]:
# plot them features! what's contributing most to our model?
# xgb.plot.multi.trees(model = bst_model)

### for submission

In [1]:
# adding values to `Ch` andd `Choice` levels to prevent errors
# data.test[,c("Ch1", "Ch2", "Ch3", "Ch4")] <- 1
# data.test[,c("Choice")] <- "Ch1"

data.test <- data.test[c('Case','No','Task','CC1','GN1','NS1','BU1', 'FA1' ,'LD1', 'BZ1', 'FC1', 'FP1', 'RP1' ,'PP1' ,
                'KA1', 'SC1', 'TS1', 'NV1', 'MA1', 'LB1' ,'AF1' ,'HU1' ,'Price1' ,'CC2' ,'GN2', 'NS2', 'BU2' ,
                'FA2', 'LD2', 'BZ2', 'FC2', 'FP2', 'RP2' ,'PP2', 'KA2' ,'SC2' ,'TS2' ,'NV2', 'MA2' ,'LB2', 'AF2' ,
                'HU2', 'Price2' ,'CC3' ,'GN3', 'NS3', 'BU3' ,'FA3', 'LD3' ,'BZ3' ,'FC3', 'FP3', 'RP3', 'PP3' ,'KA3',
                'SC3' ,'TS3' ,'NV3' ,'MA3' ,'LB3', 'AF3', 'HU3' ,'Price3', 'CC4', 'GN4', 'NS4', 'BU4' ,'FA4', 'LD4',
                'BZ4' ,'FC4' ,'FP4' ,'RP4', 'PP4' ,'KA4', 'SC4' ,'TS4' ,'NV4', 'MA4', 'LB4', 'AF4' ,'HU4', 'Price4',
                'segmentind' , 'yearind','milesind','milesa','nightind','nighta' , 'pparkind','genderind', 'ageind',
                'agea','educind', 'regionind', 'Urbind', 'incomeind', 'incomea' )]
data.test_variables <- as.matrix(subset(data.test, select= -c(Case,No,Task))) 
data.test_matrix <- xgb.DMatrix(data = data.test_variables)
data_matrix <- xgb.DMatrix(data = data_variables, label = data_label)

bst_model <- xgb.train(params = xgb_params,
                       data = data_matrix ,
                       nrounds = nround)
# Predict on test set
test_pred <- predict(bst_model, newdata = data.test_matrix,label = data_label)
test_prediction <- matrix(test_pred, nrow = 4,
                          ncol=length(test_pred)/4)

# predicting on validation set 
data.test[,c('Ch1','Ch2','Ch3','Ch4')]<-test_prediction
submission<-data.test[,c('No','Ch1','Ch2','Ch3','Ch4')]
head(submission)


ERROR: Error in eval(expr, envir, enclos): object 'data.test' not found


In [2]:
# write.csv(submission, file = "submission1.54.csv",row.names=FALSE)